In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
#from sklearn.preprocessing import OneHotEncoder
from statistics import mean

In [ ]:
data = pd.read_csv('final_modifica2.csv')

In [25]:
data.head(10)

,Unnamed: 0,id,name,producer,nation,local1,type,use,abv,degree,sweet,acidity,body,tannin,price,year,ml,latitudine,longitudine,cec,cfvo,clay,nitrogen,phh2o,sand,silt,tempmax,tempmin,temp,dew,humidity,precip,windspeed,solarradiation,solarenergy,uvindex,score
0,0,137591,"Citra, Merlot",Citra,Italy,NaN,Red,Table,13.0,17.0,1,3,4,3,0,2017,750,42.353678,14.386794,292.0,111.0,330.0,375.0,73.0,229.0,441.0,26.1615,14.6430,20.7400,13.1120,64.84950,56.89,20.3355,277.1710,23.9225,8.37,3.1
1,1,137738,"Geografico, Pulleraia",Geografico,Italy,Toscana,Red,Table,13.5,17.0,1,3,4,4,108000,2012,750,43.461670,11.430915,326.0,97.0,349.0,611.0,67.0,286.0,365.0,26.6930,14.0380,20.3960,11.1420,60.37700,420.88,17.9220,269.8290,23.3050,8.17,3.8
2,2,137960,"Castello di Ama, L'Apparita",Castello di Ama,Italy,Toscana,Red,Table,14.5,17.0,1,4,5,5,500000,2014,750,43.433653,11.400314,310.0,116.0,307.0,352.0,74.0,349.0,344.0,24.8185,13.5600,19.1435,12.3925,68.16150,335.44,16.6615,256.6615,22.1620,7.83,4.3
3,3,138381,Masseto,Ornellaia,Italy,Toscana,Red,Table,15.0,0.0,2,4,5,5,1900000,2016,750,43.234006,10.617933,233.0,102.0,293.0,365.0,75.0,356.0,352.0,25.7405,15.3620,20.5540,14.6180,71.43000,228.92,19.2610,275.1570,23.7595,8.33,4.7
4,4,139447,"Folonari, Bolgheri Campo Al Mare",Tenute Ambrogio e Giovanni Folonari,Italy,Toscana,Red,Table,14.0,17.0,1,3,4,4,0,2007,750,43.628413,11.285756,247.0,120.0,328.0,564.0,69.0,269.0,403.0,24.8735,12.2595,18.3315,10.3645,63.26340,265.14,22.1110,NaN,NaN,NaN,3.8
5,5,139495,"Maculan, Brentino",Maculan,Italy,Veneto,Red,Table,13.5,17.0,1,3,4,3,0,2006,750,45.701475,11.570923,305.0,188.0,309.0,534.0,73.0,221.0,470.0,22.3440,13.6375,17.9380,11.7600,69.94265,757.76,13.3555,NaN,NaN,NaN,3.8
6,6,139545,"Tenimenti Angelini, Tuttobene Rosso",Tenimenti Angelini,Italy,Toscana,Red,Table,13.0,17.0,1,3,3,3,0,2010,750,43.056610,11.485191,254.0,159.0,336.0,427.0,72.0,283.0,381.0,22.8795,13.2500,18.1395,11.8035,69.15100,280.69,18.7090,259.4500,22.4075,7.87,3.2
7,7,139667,Gualdo del Re l'Rennero,Gualdo del Re,Italy,Toscana,Red,Table,13.5,19.0,1,3,4,4,242000,2001,750,43.052645,10.703719,258.0,109.0,256.0,277.0,76.0,325.0,419.0,0.0000,0.0000,0.0000,0.0000,0.00000,0.00,0.0000,0.0000,0.0000,0.00,4.2
8,8,139686,"Brancaia, Il Blu",Brancaia,Italy,Toscana,Red,Table,13.0,17.0,1,4,4,4,260000,2016,750,43.490423,11.378227,311.0,94.0,302.0,497.0,71.0,356.0,342.0,26.2370,14.4570,20.2290,13.2950,67.68400,292.81,15.2460,263.8170,22.7980,7.99,4.4
9,9,139690,La Rampa di Fugnano Gisele,la Rampa di Fugnano,Italy,Toscana,Red,Table,14.0,18.0,1,3,4,4,162800,2002,750,43.460998,11.042663,273.0,83.0,308.0,333.0,77.0,303.0,390.0,24.9410,14.3725,19.5170,12.9685,68.80680,499.48,19.2305,NaN,NaN,NaN,3.9


## Preprocessing

#### One-hot-encoding categorical data ('type' and 'use' columns):

In [ ]:
one_hot_encoded_data = pd.get_dummies(data, columns = ['type','use']) #It also automatically removes the original columns

In [ ]:
#one_hot_encoded_data.head(10)

#### Change abv and degree (remove the tilde) --> if for example I find degree = 16~18, I replace it with the mean of 16 and 18

In [ ]:
data[['abv', 'degree']] = data[['abv','degree']].fillna('0') #This comes in handy when applying the following function to avoid errors

In [ ]:
data['degree']=data['degree'].astype('string')
data['abv']=data['abv'].astype('string')

In [ ]:
def remove_tilde(row):

    if "~" in row:
        x = row.split("~")
        x_int = [float(i) for i in x]
        row = mean(x_int)
    
    return row

In [ ]:
data['degree'] = data.apply(lambda row: remove_tilde(row['degree']), axis=1)
data['abv'] = data.apply(lambda row: remove_tilde(row['abv']), axis=1)

In [ ]:
data['degree']=data['degree'].astype('float')
data['abv']=data['abv'].astype('float')

In [18]:
## CHECK WHICH WINES HAVE 0 AS VALUE FOR ABV ##

#data.loc[data['abv'] == 0] #32 vini

## AND THEN COLLECT THEM BY HAND LIKE A BOSS ##

32

Per i vini a cui manca l'anno ho messo come ABV quello di un annata presa a caso


In [19]:
abv_missing = [15,14,13,14.5,13.5,14,13.5,14,14,14,15,15.5,14,13.5,14.5,13,14.5,14.5,14,13.5,13,13.5,14,14,13.5,12.5,14,13,13,13.5,14.5,0] 
i_abv = 0
def fix_abv(row): #Per aggiustare i valori di ABV
    
    global i_abv ### QUA MODIFICA LA VARIABILE GLOBALE quindi se fate attenzione se dovete riutilizzare la funzione
    if row==0:
        row = abv_missing[i_abv]
        i_abv += 1
    return row


In [20]:
#len(abv_missing)

32

In [21]:

data['abv'] = data.apply(lambda row: fix_abv(row['abv']), axis=1)

In [23]:
data.loc[data['abv'] == 0] #Noterete che manca il vino numero 143 il cui ABV è introvabile

,Unnamed: 0,id,name,producer,nation,local1,type,use,abv,degree,sweet,acidity,body,tannin,price,year,ml,latitudine,longitudine,cec,cfvo,clay,nitrogen,phh2o,sand,silt,tempmax,tempmin,temp,dew,humidity,precip,windspeed,solarradiation,solarenergy,uvindex,score
143,143,154056,"Soprano, Rosso Sweet",Soprano,Italy,NaN,Red,Table,0.0,0.0,4,1,4,2,0,2019,750,43.541129,7.953705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.5615,16.8555,19.5475,13.8855,71.316,241.73,19.216,211.753,18.2855,7.39,3.1


Bisognerebbe fare lo stesso procedimento per il degree che (*drum roll*) è la cristo di temperatura consigliata per bere il vino ed è un po' più difficile da trovare (mancano 33 records)

In [ ]:
#data.loc[data['degree'] == 0] #33 vini


### - There are a few wines with year = 0 (what to do with them?)
### - Also a few NaN regarding "solarradiation" and "solarenergy" which I'm not too sure how to handle 
### - And apparently a few wonky records about the temperature